In [134]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModel
import torchaudio
from torch.utils.data import DataLoader
import os
import sys
sys.path.append("..")
from src.evaluation.representations import compute_linear_eval


In [136]:
data_folder = "../data/speechcommands"

# Load the full dataset (v0.02 with 35 classes)
import torchaudio
dataset = torchaudio.datasets.SPEECHCOMMANDS(
    root=data_folder,
    download=True,
)


def collate_fn(batch):

    word_dict = {
        "yes": 0,
        "no": 1,
        "up": 2,
        "down": 3,
        "left": 4,
    }
    """
    Processes a batch of samples to make waveforms uniform in length.
    A batch is a list of tuples: (waveform, sample_rate, label, speaker_id, utterance_number)
    """
    target_length = 16000  # 1 second at 16kHz
    waveforms = []
    labels = []

    # Process each sample in the batch
    for waveform, _, label, _, _ in batch:
        # Pad the waveform if it's shorter than the target length
        if waveform.shape[1] < target_length:
            padding_needed = target_length - waveform.shape[1]
            padding = torch.zeros((1, padding_needed))
            waveform = torch.cat([waveform, padding], dim=1)
        # Truncate the waveform if it's longer
        else:
            waveform = waveform[:, :target_length]
        

        waveforms.append(waveform)
        labels.append(label) # You'll likely want to convert these strings to numbers later

    # Stack the processed waveforms into a single tensor
    waveforms_tensor = torch.stack(waveforms)
    
    return waveforms_tensor.squeeze(1), labels

# When creating your DataLoader, pass your custom function
# Assuming 'dataset' is your torchaudio.datasets.SPEECHCOMMANDS object
dataloader = DataLoader(
    dataset, 
    batch_size=100, 
    shuffle=False, 
    collate_fn=collate_fn  # This is the key part
)




100%|██████████| 2.26G/2.26G [00:54<00:00, 44.7MB/s]


In [132]:
dataset.


AttributeError: 'SPEECHCOMMANDS' object has no attribute 'labels'

In [65]:
from transformers import Wav2Vec2Model
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")

device = 'mps'
model.to(device)
labels = []
representations = []
from tqdm import tqdm
with torch.no_grad():
    for batch in tqdm(dataloader):
        waveform, label = batch
        
        labels += label
        representations.append(model(waveform.to(device)).last_hidden_state.cpu())
representations = torch.cat(representations, dim=0)


/Users/alexkaravos/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/alexkaravos/anaconda3/lib/python3.11/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/Users/alexkaravos/anaconda3/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Some weights of the model checkpoint at facebook/

In [67]:
feat_mean,feat_std = representations.mean(dim=1),representations.std(dim=1)
cat_feat = torch.cat([feat_mean,feat_std],dim=1)


In [84]:
speech_labels = [
    'backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow',
    'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no',
    'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree',
    'two', 'up', 'visual', 'wow', 'yes', 'zero'
]
int_labels = [speech_labels.index(label) for label in labels]
int_labels = torch.tensor(int_labels).int()










In [122]:
dataset_len = len(int_labels)
train_len = int(dataset_len * 0.8)
test_len = int(dataset_len * 0.2)

idx = torch.randperm(dataset_len)
train_idx = idx[:train_len]
test_idx = idx[train_len:]

train_representations = feat_mean[train_idx]
test_representations = feat_mean[test_idx]
train_labels = int_labels[train_idx]
test_labels = int_labels[test_idx]

train_dataset = torch.utils.data.TensorDataset(train_representations, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_representations, test_labels)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)


In [123]:
len(test_idx)

21166

In [125]:
from torch import nn

class LinearClassifier(nn.Module):
    def __init__(self,input_dim,num_classes):
        super(LinearClassifier,self).__init__()
        self.linear = nn.Linear(input_dim,num_classes)
    
    def forward(self,x):
        return self.linear(x)
    
class AttentionClassifier(nn.Module):
    def __init__(self,input_dim,num_classes):
        super(AttentionClassifier,self).__init__()
        self.cls_token = nn.Parameter(torch.randn(1, 1, input_dim))
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=input_dim,
            nhead=8,
            dim_feedforward=1536,
            dropout=0.1,
            batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=3)
        self.classifier = nn.Linear(input_dim, num_classes)
        
    def forward(self, x):
        # x shape: (bs, 49, 768)
        batch_size = x.size(0)
        
        # Expand cls_token for batch
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # (bs, 1, 768)
        
        # Use input sequence as memory for decoder
        memory = x  # (bs, 49, 768)
        
        # Pass cls_token through transformer decoder
        decoded = self.transformer_decoder(cls_tokens, memory)  # (bs, 1, 768)
        
        # Extract cls token and classify
        cls_output = decoded.squeeze(1)  # (bs, 768)
        logits = self.classifier(cls_output)  # (bs, 35)
        
        return logits

model = LinearClassifier(768,35)
model.to('mps')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
from tqdm import tqdm

for epoch in range(30):
    lossval = 0   
    epoch_bar = tqdm(train_dataloader,desc=f'Epoch {epoch+1}')  
    for batch in epoch_bar:
        features, labels = batch
        optimizer.zero_grad()
        outputs = model(features.to('mps'))
        loss = criterion(outputs, labels.to('mps'))
        loss.backward()
        lossval += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        epoch_bar.set_postfix({'Loss': f'{loss.item():.4f}'})










Epoch 30: 100%|██████████| 85/85 [00:00<00:00, 115.83it/s, Loss=0.6889]


In [126]:


model.eval()
predictions = []
labels = []
with torch.no_grad():
    for batch in test_dataloader:
        features, label = batch
        
        predictions.append(model(features.to('mps')).cpu().argmax(dim=1))
        labels.append(label)

predictions = torch.cat(predictions, dim=0)
labels = torch.cat(labels, dim=0)

accuracy = (predictions == labels).float().mean()
print(f"Accuracy: {accuracy:.4f}")





Accuracy: 0.8281


In [ ]:
from torchaudio import transforms

In [133]:
#get the pca of the cat features

from sklearn.decomposition import PCA

pca = PCA(n_components=20)
pca.fit(cat_feat)
pca_feat = pca.transform(cat_feat)

from sklearn.metrics import normalized_mutual_info_score

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=35,random_state=0)
kmeans.fit(pca_feat)

print(normalized_mutual_info_score(kmeans.labels_,int_labels))

0.12074510112527835
